In [1]:
import json
import pandas as pd
import warnings
import seaborn as sns
from pyprojroot import here
import plotly.express as px

methods = ["JustCopy", "TimeGAN", "Time-Transformer", "TransFusion", "TTS-GAN", "TimeVQVAE"]
datasets = ["D2", "D3", "D4", "D5", "D6", "D7"]

rows = []

for method in methods:
    for dataset in datasets:
        file_candidates = list(here('result').glob(f'numeric_{method}_{dataset}_*.json'))
        if len(file_candidates) != 1:
            warnings.warn(f"Ignoring {method} {dataset}: Expected one result file for {method} {dataset}, instead matched {file_candidates}.")
            continue
        file_path = file_candidates[0]
        
        with open(file_path) as f:
            data = json.load(f)
        
        rows.append(((method, dataset), data))

df_all = pd.DataFrame.from_dict(dict(rows), orient="index")
df_all.index = pd.MultiIndex.from_tuples(df_all.index, names=["Method", "Dataset"])

# load timings
def timings_path(method):
    paths = list(here("models").glob(f"*{method}*/timings.csv"))
    assert len(paths) == 1
    return paths[0]

timings = pd.DataFrame()
for method in methods:
    df = pd.read_csv(timings_path(method))
    df["Method"] = method
    timings = pd.concat([timings, df], ignore_index=True)

_timings = timings.melt(id_vars="Method", var_name="Dataset", value_name="Time").set_index(["Method", "Dataset"]).sort_index(level="Method")

df_all = pd.concat([df_all, _timings], axis = 1)

In [2]:
cm = sns.diverging_palette(h_neg=130, h_pos=0, as_cmap=True)

def style_results_table(df):
    return df.style\
    .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
    .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
    .background_gradient(cmap=cm, vmin=0, vmax=15, subset=["DTW"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1000, subset=["Time"])\
    .format({"DTW": "{:.2e}", "C-FID": "{:.2e}", "ED": "{:.2e}"})

# Note: 
# Some methods were trained on old D7 and or evaluated on old D7
df_no_copy = df_all.drop(index="JustCopy", level="Method")
style_results_table(df_all)

In [3]:
style_results_table(df_all.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"]))

## Spider Plots

In [21]:
rankings = df_no_copy.unstack(level=1).rank()

ranking_by_metric = rankings.stack().reset_index().groupby("Method").mean()
spider_by_metric = ranking_by_metric.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_metric, r="value", theta="variable", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [5.9, 1], "dtick": 1}})

In [22]:
ranking_by_dataset = rankings.stack(level=0).reset_index().groupby("Method").mean()
spider_by_dataset = ranking_by_dataset.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_dataset, r="value", theta="Dataset", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [5.9,1], "dtick": 1}})

In [23]:
import plotly.graph_objects as go


def plot_average_rankings(df, title, y_offsets= [1, -1, 1, -1, 1, -1]):
    s = df.unstack(level=1).rank().mean(axis=1)

    colors = [color for _, color in zip(methods, px.colors.qualitative.Plotly)]


    fig = go.Figure(go.Scatter(
        x=s.values,
        y=[0]*len(s),
        mode="markers+text",
        text=s.index,
        textfont=dict(color=colors),
        textposition=["top center" if y>0 else "bottom center" for y in y_offsets],
        marker=dict(size=12, color=colors)
    ))

    range_max = len(df.index.get_level_values(level="Method").unique())
    print(range_max)

    # Thin horizontal number line
    fig.add_shape(type="line",
                x0=1, x1=range_max,
                y0=0, y1=0,
                line=dict(color="black", width=1),
                layer="below")

    # Layout tweaks for minimal look
    fig.update_yaxes(visible=False)
    fig.update_xaxes(range=[0.8, range_max +0.2], showgrid=True, zeroline=False)
    fig.update_layout(
        height=200,
        xaxis_title="Average Rank",
        yaxis_title="",
        showlegend=False,
        margin=dict(t=60, b=60, l=150, r=150),
        title=title,
    )

    fig.show()

plot_average_rankings(df_no_copy, "All Measures", [-1, 1, -1, -1, 1])


5


In [24]:
import scikit_posthocs as sp

def conover_test(df):
    df = df.unstack(level=1).rank().T.reset_index().melt(id_vars=['level_0', 'Dataset'], var_name='Method', value_name='Rank')
    posthoc = sp.posthoc_conover(df, group_col='Method', val_col='Rank', p_adjust='holm')

    def highlight_below_threshold(val):
        color = 'background-color: turquoise' if val < 0.01 else ''
        return color

    return posthoc.style.applymap(highlight_below_threshold).format("{:.2e}")

conover_test(df_no_copy)

,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,5.42e-21,7.00e-18,3.41e-26,1.85e-38
Time-Transformer,5.42e-21,1.00e+00,3.33e-01,2.55e-01,7.61e-06
TimeGAN,7.00e-18,3.33e-01,1.00e+00,3.90e-02,6.70e-08
TimeVQVAE,3.41e-26,2.55e-01,3.90e-02,1.00e+00,3.29e-03
TransFusion,1.85e-38,7.61e-06,6.70e-08,3.29e-03,1.00e+00


In [8]:
plot_average_rankings(df_all.drop(index="JustCopy", level="Method").drop(columns=["PS", "Time"]), "Fidelity")

5


In [25]:
conover_test(df_no_copy.drop(columns=["PS", "Time"]))

,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,5.52e-19,1.38e-20,3.52e-27,5.36e-39
Time-Transformer,5.52e-19,1.00e+00,5.94e-01,2.99e-02,1.85e-08
TimeGAN,1.38e-20,5.94e-01,1.00e+00,8.02e-02,2.59e-07
TimeVQVAE,3.52e-27,2.99e-02,8.02e-02,1.00e+00,1.77e-03
TransFusion,5.36e-39,1.85e-08,2.59e-07,1.77e-03,1.00e+00


In [26]:
plot_average_rankings(df_no_copy.drop(columns=["Time"]), "Without Timing")

5


In [27]:
conover_test(df_no_copy.drop(columns=["Time"]))

,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,2.74e-19,1.91e-22,3.42e-27,5.62e-43
Time-Transformer,2.74e-19,1.00e+00,3.23e-01,4.94e-02,2.09e-10
TimeGAN,1.91e-22,3.23e-01,1.00e+00,3.11e-01,4.84e-08
TimeVQVAE,3.42e-27,4.94e-02,3.11e-01,1.00e+00,4.05e-05
TransFusion,5.62e-43,2.09e-10,4.84e-08,4.05e-05,1.00e+00


In [28]:
plot_average_rankings(df_no_copy["PS"], "Predictive Score", [1, 1, 1, -1, 1])

5


In [29]:
conover_test(df_no_copy["PS"].to_frame())

,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,3.79e-01,2.41e-02,3.79e-01,1.82e-04
Time-Transformer,3.79e-01,1.00e+00,5.31e-01,1.00e+00,2.41e-02
TimeGAN,2.41e-02,5.31e-01,1.00e+00,5.31e-01,3.79e-01
TimeVQVAE,3.79e-01,1.00e+00,5.31e-01,1.00e+00,2.41e-02
TransFusion,1.82e-04,2.41e-02,3.79e-01,2.41e-02,1.00e+00


In [30]:
from scipy.stats import friedmanchisquare

df = df_no_copy.unstack(level=1).T
ranks = df.rank(axis=0, method='average', ascending=False)

stat, p = friedmanchisquare(*[ranks.loc[m].values for m in ranks.index])
p

6.268300811770057e-19

In [31]:
import pandas as pd

def leaderboard(df):
    out = pd.DataFrame(index=df.index.levels[1], columns=df.columns)

    for lvl2 in df.index.levels[1]:
        subset = df.xs(lvl2, level=1)
        min_idx = subset.idxmin()
        out.loc[lvl2] = min_idx


    def color_cells(value):
        if value == "TTS-GAN":
            return 'background-color: yellow'
        if value == "TransFusion":
            return 'background-color: lightgreen'
        if value == "TimeGAN":
            return 'background-color: lightblue'
        if value == "Time-Transformer":
            return 'background-color: pink'
        if value == "TimeVQVAE":
            return 'background-color: turquoise'

    return out.style.applymap(color_cells)

leaderboard(df_no_copy)

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer
D3,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D4,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer,Time-Transformer,Time-Transformer
D5,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,Time-Transformer,Time-Transformer,Time-Transformer
D6,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D7,TransFusion,TimeGAN,TransFusion,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer


In [16]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["TTS-GAN", "TimeGAN"])])

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D3,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D4,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D5,TimeGAN,TTS-GAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D6,TTS-GAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D7,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN


In [17]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["Time-Transformer", "TimeGAN"])])

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer
D3,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D4,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D5,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D6,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D7,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,TimeGAN,Time-Transformer,TimeGAN,TimeGAN,Time-Transformer


In [18]:
rankings

DS                            PS                 ...  DTW  \
Dataset            D2   D3   D4   D5   D6   D7   D2   D3   D4   D5  ...   D4   
Method                                                              ...        
TTS-GAN           5.0  5.0  5.0  5.0  4.0  4.5  5.0  5.0  5.0  2.0  ...  5.0   
Time-Transformer  4.0  4.0  3.0  4.0  4.0  3.0  4.0  4.0  3.0  4.0  ...  1.0   
TimeGAN           1.0  2.0  2.0  2.0  4.0  2.0  1.0  2.0  4.0  3.0  ...  4.0   
TimeVQVAE         3.0  3.0  4.0  3.0  2.0  4.5  3.0  3.0  2.0  5.0  ...  2.0   
TransFusion       2.0  1.0  1.0  1.0  1.0  1.0  2.0  1.0  1.0  1.0  ...  3.0   

                                Time                           
Dataset            D5   D6   D7   D2   D3   D4   D5   D6   D7  
Method                                                         
TTS-GAN           5.0  5.0  5.0  5.0  4.0  4.0  5.0  3.0  3.0  
Time-Transformer  1.0  2.0  3.0  1.0  1.0  1.0  1.0  1.0  1.0  
TimeGAN           4.0  4.0  2.0  4.0  5.0  5.0  4.0  4.0  5.0  
TimeVQVAE         3.0  1.0  4.0  3.0  2.0  2.0  2.0  2.0  2.0  
TransFusion       2.0  3.0  1.0  2.0  3.0  3.0  3.0  5.0  4.0  

[5 rows x 60 columns]

In [32]:
df = df_no_copy.unstack(level=1).rank()
utility_cols = [col for col in df.columns if col[0] in ["PS"]]

fidelity_cols = [col for col in df.columns if col[0] not in ["PS", "Time"]]

timing_cols = [col for col in df.columns if col[0] in ["Time"]]

avg_utility = df[utility_cols].mean(axis=1)
avg_fidelity = df[fidelity_cols].mean(axis=1)
avg_timing = df[timing_cols].mean(axis=1)

new_df = pd.DataFrame({
    "Utility": avg_utility,
    "Fidelity": avg_fidelity,
    "Time": avg_timing
})

new_df.style.background_gradient(cmap=cm)

,Utility,Fidelity,Time
Method,,,
TTS-GAN,4.500000,4.906250,4.000000
Time-Transformer,3.333333,2.979167,1.000000
TimeGAN,2.500000,2.875000,4.500000
TimeVQVAE,3.333333,2.468750,2.166667
TransFusion,1.333333,1.770833,3.333333


In [35]:
timings.set_index("Method").drop(index="JustCopy")

,D2,D3,D4,D5,D6,D7
Method,,,,,,
TimeGAN,259,1373,1195,304,1252,1385
Time-Transformer,13,16,28,66,77,55
TransFusion,43,222,454,201,1387,856
TTS-GAN,646,660,649,649,660,645
TimeVQVAE,73,94,94,74,88,86
